In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.4
SparkUI available at http://10.1.1.170:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.35-bd2a3a9fe07c
LOGGING: writing to /home/brendan/code/VariantSpark/dev-notebooks/hail-20210103-1502-0.2.35-bd2a3a9fe07c.log


In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [3]:
data = hl.import_vcf('../data/hipsterIndex/hipster.vcf.bgz')

In [4]:
data.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AA: str, 
        AC: array<int32>, 
        AF: array<float64>, 
        AFR_AF: array<float64>, 
        AMR_AF: array<float64>, 
        AN: int32, 
        CIEND: array<int32>, 
        CIPOS: array<int32>, 
        CS: str, 
        DP: int32, 
        EAS_AF: array<float64>, 
        END: int32, 
        EUR_AF: array<float64>, 
        EX_TARGET: bool, 
        IMPRECISE: bool, 
        MC: array<str>, 
        MEINFO: array<str>, 
        MEND: int32, 
        MLEN: int32, 
        MSTART: int32, 
        MULTI_ALLELIC: bool, 
        NS: int32, 
        SAS_AF: array<float64>, 
        SVLEN: array<int32>, 
        SVTYPE: str, 
        TSD: str, 
      

In [5]:
labels = hl.import_table('../data/hipsterIndex/hipster_labels.txt', delimiter=',', 
                types=dict(label='int64', score='float64')).key_by('samples')

2021-01-03 15:02:42 Hail: INFO: Reading table with no type imputation
  Loading column 'samples' as type 'str' (type not specified)
  Loading column 'score' as type 'float64' (user-specified)
  Loading column 'label' as type 'int64' (user-specified)



In [6]:
labels.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'samples': str 
    'score': float64 
    'label': int64 
----------------------------------------
Key: ['samples']
----------------------------------------


In [7]:
mt = data.annotate_cols(label = labels[data.s])
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'label': struct {
        score: float64, 
        label: int64
    }
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AA: str, 
        AC: array<int32>, 
        AF: array<float64>, 
        AFR_AF: array<float64>, 
        AMR_AF: array<float64>, 
        AN: int32, 
        CIEND: array<int32>, 
        CIPOS: array<int32>, 
        CS: str, 
        DP: int32, 
        EAS_AF: array<float64>, 
        END: int32, 
        EUR_AF: array<float64>, 
        EX_TARGET: bool, 
        IMPRECISE: bool, 
        MC: array<str>, 
        MEINFO: array<str>, 
        MEND: int32, 
        MLEN: int32, 
        MSTART: int32, 
        MULTI_ALLELIC: bool, 
        NS: int32, 
        SAS_AF: array<float64>, 
   

In [8]:
mt.count()

2021-01-03 15:02:43 Hail: INFO: Coerced almost-sorted dataset


(17010, 2504)

In [9]:
gwas = hl.linear_regression_rows(y=mt.label.score,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])

2021-01-03 15:02:45 Hail: INFO: Coerced almost-sorted dataset
2021-01-03 15:02:46 Hail: INFO: linear_regression_rows: running on 2504 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


In [10]:
gwas.show()

locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh37>,array<str>,int32,float64,float64,float64,float64,float64,float64
2:109511398,"[""G"",""A""]",2504,4.00e+00,3.50e+01,-7.03e-01,1.62e+00,-4.34e-01,6.64e-01
2:109511454,"[""C"",""A""]",2504,1.00e+00,1.15e+01,2.05e+00,3.24e+00,6.33e-01,5.27e-01
2:109511463,"[""G"",""A""]",2504,1.47e+02,1.30e+03,-5.79e-01,2.60e-01,-2.23e+00,2.61e-02
2:109511467,"[""GACTC"",""G""]",2504,5.47e+02,5.06e+03,-2.07e-01,1.42e-01,-1.45e+00,1.47e-01
2:109511478,"[""C"",""T""]",2504,1.00e+00,1.20e+01,2.55e+00,3.24e+00,7.88e-01,4.31e-01
2:109511497,"[""G"",""T""]",2504,1.00e+00,1.05e+01,1.05e+00,3.24e+00,3.24e-01,7.46e-01
2:109511525,"[""G"",""GAATT""]",2504,1.30e+01,9.95e+01,-1.81e+00,8.99e-01,-2.01e+00,4.45e-02
2:109511527,"[""A"",""C""]",2504,1.00e+00,5.50e+00,-3.95e+00,3.24e+00,-1.22e+00,2.22e-01
2:109511532,"[""A"",""G""]",2504,1.00e+00,6.50e+00,-2.95e+00,3.24e+00,-9.12e-01,3.62e-01


In [11]:
p = hl.plot.manhattan(gwas.p_value)
show(p)